## OSC SERVER

In [1]:
"""Small example OSC server

This program listens to several addresses, and prints some information about
received packets.
"""
import argparse
import math

from pythonosc.dispatcher import Dispatcher
from pythonosc import osc_server

def print_volume_handler(unused_addr, args, volume):
  print("[{0}] ~ {1}".format(args[0], volume))

def print_compute_handler(unused_addr, args, volume):
  try:
    print("[{0}] ~ {1}".format(args[0], args[1](volume)))
  except ValueError: pass

if __name__ == "__main__":
  parser = argparse.ArgumentParser()
  parser.add_argument("--ip",
      default="127.0.0.1", help="The ip to listen on")
  parser.add_argument("--port",
      type=int, default=5005, help="The port to listen on")
  args = parser.parse_args()

  dispatcher = Dispatcher()
  dispatcher.map("/filter", print)
  dispatcher.map("/volume", print_volume_handler, "Volume")
  dispatcher.map("/logvolume", print_compute_handler, "Log volume", math.log)

  server = osc_server.ThreadingOSCUDPServer(
      (args.ip, args.port), dispatcher)
  print("Serving on {}".format(server.server_address))
  server.serve_forever()

usage: ipykernel_launcher.py [-h] [--ip IP] [--port PORT]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/reini/.local/share/jupyter/runtime/kernel-ef77dd46-193f-43a6-89fb-6ce6b8d92dac.json


SystemExit: 2

/home/reini/PycharmProjects/AMPEVock_MuteButton/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## OSC Client

In [ ]:
"""Small example OSC client

This program sends 10 random values between 0.0 and 1.0 to the /filter address,
waiting for 1 seconds between each value.
"""
import argparse
import random
import time

from pythonosc import udp_client


if __name__ == "__main__":
  parser = argparse.ArgumentParser()
  parser.add_argument("--ip", default="127.0.0.1",
      help="The ip of the OSC server")
  parser.add_argument("--port", type=int, default=5005,
      help="The port the OSC server is listening on")
  args = parser.parse_args()

  client = udp_client.SimpleUDPClient(args.ip, args.port)

  for x in range(10):
    client.send_message("/filter", random.random())
    time.sleep(1)

## Reini Class Test Clientimport argparse


In [3]:
import argparse
from pythonosc import udp_client

class OSCClient:

    def __init__(self, ip: str = "127.0.0.1", port: int = 5005):
        self.ip = ip
        self.port = port
        

        parser = argparse.ArgumentParser()
        parser.add_argument("--ip", default=ip, help="IP address of OSC server")
        parser.add_argument("--port", default=port, type=int, help="Port of OSC server")
        args = parser.parse_args()

        self.client = udp_client.SimpleUDPClient(args.ip, args.port)

    def send_message(self, address: str, message: str):
        self.client.send_message(address, message)

if __name__ == '__main__':

    oscclient = OSCClient()

    oscclient.send_message(address="/ip", message="Hello World")


usage: ipykernel_launcher.py [-h] [--ip IP] [--port PORT]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/reini/.local/share/jupyter/runtime/kernel-ef77dd46-193f-43a6-89fb-6ce6b8d92dac.json


SystemExit: 2

/home/reini/PycharmProjects/AMPEVock_MuteButton/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Building Bundles Test

In [1]:
from pythonosc import osc_bundle_builder
from pythonosc import osc_message_builder

bundle = osc_bundle_builder.OscBundleBuilder(
    osc_bundle_builder.IMMEDIATELY)
msg = osc_message_builder.OscMessageBuilder(address="/SYNC")
msg.add_arg(4.0)
# Add 4 messages in the bundle, each with more arguments.
bundle.add_content(msg.build())
msg.add_arg(2)
bundle.add_content(msg.build())
msg.add_arg("value")
bundle.add_content(msg.build())
msg.add_arg(b"\x01\x02\x03")
bundle.add_content(msg.build())

sub_bundle = bundle.build()
# Now add the same bundle inside itself.
bundle.add_content(sub_bundle)
# The bundle has 5 elements in total now.

bundle = bundle.build()
# You can now send it via a client as described in other examples.